# import packages

In [1]:
import sys
import os
import io
# os.environ["CUDA_VISIBLE_DEVICES"] = '1'
import gc
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import torch
from PIL import Image
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import recall_score, roc_auc_score
import sys
import timm
from timm import create_model, list_models
from timm.data import create_transform
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
import random
import wandb
from wandb import AlertLevel


import torchvision
from torch.utils.data import Dataset
from torch.nn import functional as F
from torch.nn import Module, Linear, Sequential, ModuleList, ReLU, Dropout, Flatten
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import Adam, SGD, AdamW, lr_scheduler
import warnings
warnings.filterwarnings('ignore')


# os.environ['WANDB_API_KEY'] = 'YOUR KEY HERE' # I dont provide this, but I set it to offline.
gc.collect()
torch.cuda.empty_cache()

from utils import seed_everything, init_logger, get_timediff, optimal_f1, gc_collect, add_weight_decay, get_parameter_number
from model import GeM, BreastCancerModel
from dataset import BreastCancerDataSet_16bit, BreastCancerDataSet_8bit, mixup_augmentation, get_transforms_8bit, get_transforms_16bit

# CFG

In [2]:
class CFG:
    image_size =  (1536, 896)
    tta = True
    
    seed = 1788
    num_workers = 5
    valid_batch_size = 32
    gpu_parallel = False
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    df_path = f'df_train_0001.csv'

    normalize_mean= [0.485, 0.456, 0.406]  # [0.21596, 0.21596, 0.21596]
    normalize_std = [0.229, 0.224, 0.225]  # [0.18558, 0.18558, 0.18558]
    positive_target_weight = 1
    target = 'cancer'
    ensemble = True
    n_folds = 5
    folds = [0]
    accum_iter=1


comp_data_dir = '/blue/eel6825/f.yang1/kaggle/'
images_dir = f'/blue/eel6825/f.yang1/kaggle/train_images/'
output_dir = f'/blue/eel6825/f.yang1/'
os.makedirs(output_dir, exist_ok=True)

DEBUG = True
WANDB_SWEEP = False
TRAIN = True
CV = True

In [3]:
seed_everything(CFG.seed)

In [4]:
# try:
#     df_train = pd.read_csv(CFG.df_path)
# except:
df_train = pd.read_csv(f'{comp_data_dir}/train.csv')
split = StratifiedGroupKFold(CFG.n_folds)
for k, (_, test_idx) in enumerate(split.split(df_train, df_train.cancer, groups=df_train.patient_id)):
    df_train.loc[test_idx, 'split'] = k
df_train.split = df_train.split.astype(int)
df_train["sample_rand"] = np.random.rand(len(df_train)) 
df_train.loc[df_train["cancer"]==1, "sample_rand"] = 0.0

df_train = df_train[df_train['image_id'].astype(str) != '1942326353'].reset_index(drop=True)
df_train = df_train[df_train['patient_id'].astype(str) != '27770'].reset_index(drop=True)

# dataset class

In [5]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, 
    CenterCrop, Resize, RandomCrop, GaussianBlur, JpegCompression, Downscale, ElasticTransform, Affine, ToFloat
)
import albumentations as A
from albumentations.pytorch import ToTensorV2

def get_transforms(*, data):
    if data == 'train':
        return Compose([
            Normalize(mean=CFG.normalize_mean, std=CFG.normalize_std,),
            ToTensorV2(),
            ])
        
    elif data == 'valid':
        return Compose([
            ToFloat(max_value=255.0),
            Resize(CFG.image_size[0], CFG.image_size[1]),
            ToTensorV2(),
        ])

In [6]:
class BreastCancerDataSet(Dataset):
    def __init__(self, df, path, transforms=None):
        super().__init__()
        self.df = df
        self.path = path
        self.transforms = transforms

    def __getitem__(self, i):
        path = f'{self.path}/{self.df.iloc[i].patient_id}/{self.df.iloc[i].image_id}.jpg'
        try:
            # img = Image.open(path).convert('RGB')
            # 16bit
            img = Image.open(path)
            img = np.array(img).astype(np.uint8)
            if img.ndim == 2:
                img = np.repeat(img[:, :, np.newaxis], 3, axis=-1)
        except Exception as ex:
            print(path, ex)
            return None

        # if self.transforms:
        img = self.transforms(image=np.array(img))["image"]

        if CFG.target in self.df.columns:
            cancer_target = torch.as_tensor(self.df.iloc[i].cancer)
            return img, cancer_target

        return img

    def __len__(self):
        return len(self.df)

# model

In [7]:
class BreastCancerModel(Module):
    def __init__(self, model_arch, dropout=0.):
        super().__init__()
        self.model = create_model(
            model_arch, 
            pretrained=True, 
            num_classes=0, 
            drop_rate=dropout,
            global_pool="", 
            )
        self.num_feats = self.model.num_features

        self.cancer_logits = Linear(self.num_feats, 1)
        
        self.fc_dropout = nn.Dropout(0)
        self.global_pool = GeM(p_trainable=True)

    def forward(self, x):
        x = self.model(x) # (bs, num_feats) /  (bs, num_feats, 16, 16)
        x = self.global_pool(x) # (bs, num_feats, 1, 1)
        x = x[:,:,0,0] # # (bs, num_feats)
        cancer_logits = self.cancer_logits(self.fc_dropout(x)).squeeze() # (bs)
        return cancer_logits

# train and test

In [8]:
def load_model(path, backbone, model=None):
    state_dict = torch.load(path, map_location=CFG.device)
    if model is None:
        model = BreastCancerModel(backbone)
    model.load_state_dict(state_dict['model'])
    print(f"load model:{backbone}, thres:{state_dict['threshold']}, ")
    return model, state_dict['threshold'], state_dict['model_arch']


In [9]:
def valid_one_epoch(model, dataloader):
    model = model.to(CFG.device)
    cancer_pred_list = []
    with torch.no_grad():
        model.eval()
        losses = []; targets = []
        with tqdm(dataloader, desc='Eval', mininterval=30) as progress:
            for i, (X, y_c) in enumerate(progress):
                with autocast(enabled=True):
                    X = X.to(CFG.device)
                    y_c = y_c.to(float).to(CFG.device)
                    
                    pred_c = model(X).view(-1)
                    if CFG.tta:
                        pred_c2 = model(torch.flip(X, dims=[-1])) # horizontal mirror
                        pred_c = (pred_c + pred_c2) / 2

                    loss = F.binary_cross_entropy_with_logits(pred_c, y_c, pos_weight=torch.tensor([CFG.positive_target_weight]).to(CFG.device)).item()
                    loss = loss / CFG.accum_iter
                    
                    cancer_pred_list.append(torch.sigmoid(pred_c))
                    losses.append(loss); targets.append(y_c.cpu().numpy())
        
        targets = np.concatenate(targets)
        pred = torch.concat(cancer_pred_list).cpu().numpy()
        pf1, thres = optimal_f1(targets, pred)
        
        # Compute recall and AUC
        recall = recall_score(targets, (pred > thres).astype(int))
        
        #      (best_pf1, best_thres)  pred_value  mean_all_losses  recall  auc
        return (pf1,      thres),      pred,       np.mean(losses), recall

In [10]:
def gen_predictions(models, df_train, folds):
    df_train_predictions = []
    for model, fold in zip(models, folds):
        ds_valid = BreastCancerDataSet_8bit(df_train.query('split == @fold'), images_dir, CFG.target, get_transforms_8bit('valid', CFG.image_size, CFG.normalize_mean, CFG.normalize_std))
        valid_dataloader  = DataLoader(ds_valid, batch_size=CFG.valid_batch_size, shuffle=False, num_workers=CFG.num_workers, pin_memory=False)
        (pf1, thres), pred_cancer = valid_one_epoch(model, valid_dataloader)[:2]
        
        df_pred = pd.DataFrame(data=pred_cancer, columns=['cancer_pred_proba'])
        df_pred['cancer_pred'] = df_pred["cancer_pred_proba"] > thres
        

        df = pd.concat(
            [df_train.query('split == @fold').reset_index(drop=True), df_pred],
            axis=1
        ).sort_values(['patient_id', 'image_id'])
        df_train_predictions.append(df)
    df_train_predictions = pd.concat(df_train_predictions)
    return df_train_predictions

In [11]:
model_paths = [
        "/blue/eel6825/f.yang1/0003/0003_tf_efficientnetv2_s_FT_f0_ep4.pth", 
        "/blue/eel6825/f.yang1/0003/0003_convnext_nano_FT_f0_ep3.pth", 
        "/blue/eel6825/f.yang1/0003/0003_tf_efficientnet_b4_FT_f0_ep4.pth", 
]

backbones = [
    "tf_efficientnetv2_s",
    "convnext_nano",
    "tf_efficientnet_b4",
]

folds = [0,0,0]

assert len(model_paths) == len(folds) == len(backbones), f"got folds:{len(folds)}, model_paths:{len(model_paths)},  backbones:{len(backbones)}"


models = []

for m_path, backbone in zip(model_paths, backbones):
    model, thres, model_arch = load_model(m_path, backbone)
    model = model.to(CFG.device)
    models.append(model)

load model:tf_efficientnetv2_s, thres:0.49, 
load model:convnext_nano, thres:0.29, 
load model:tf_efficientnet_b4, thres:0.4, 


## max

In [55]:
def ensemble_predictions(models, df_train, folds):
    df_train_predictions = []
    for model in models:
        df_train_predictions.append(gen_predictions([model], df_train, folds))
    df_ensemble = pd.concat(df_train_predictions)
    df_ensemble_mean = df_ensemble.groupby(['patient_id', 'image_id'])['cancer_pred_proba'].max().reset_index()
    df_ensemble_vote = df_ensemble.groupby(['patient_id', 'image_id'])['cancer_pred'].max().reset_index()
    df_ensemble = pd.concat([df_ensemble_mean, df_ensemble_vote['cancer_pred']], axis=1)
    return df_ensemble

In [56]:
ensemble_table = ensemble_predictions(models,df_train,CFG.folds)
ensemble_table

Eval: 100%|██████████| 342/342 [05:23<00:00,  1.06it/s]


,patient_id,image_id,cancer_pred_proba,cancer_pred
0,25,822390278,0.098083,False
1,25,1442180348,0.120972,False
2,25,1723979573,0.217041,False
3,25,1789648218,0.145264,False
4,25,1962456803,0.367676,True
...,...,...,...,...
10924,65482,1203545478,0.017441,False
10925,65525,308255045,0.020920,False
10926,65525,751035345,0.016403,False
10927,65525,782942436,0.020844,False


In [57]:
df_combined = pd.merge(df_train, ensemble_table, on=['patient_id', 'image_id'])

df_clean = df_combined.dropna(subset=['cancer_pred_proba'])
df_clean

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,split,sample_rand,cancer_pred_proba,cancer_pred
0,2,10048,964141995,L,MLO,62.0,0,0,0,NaN,0,NaN,29,False,0,0.892034,0.017242,False
1,2,10048,1234933874,L,CC,62.0,0,0,0,NaN,0,NaN,29,False,0,0.111358,0.018616,False
2,2,10048,1577142909,R,MLO,62.0,0,0,0,NaN,0,NaN,29,False,0,0.196719,0.015671,False
3,2,10048,1842203124,R,CC,62.0,0,0,0,NaN,0,NaN,29,False,0,0.387039,0.016663,False
4,1,10049,349510516,L,CC,52.0,0,0,0,NaN,0,C,49,False,0,0.219363,0.020889,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10924,1,997,1775983513,R,MLO,42.0,0,0,0,1.0,0,B,49,False,0,0.023726,0.056030,False
10925,1,9989,63473691,L,MLO,60.0,0,0,0,NaN,0,C,216,False,0,0.856387,0.014175,False
10926,1,9989,1078943060,L,CC,60.0,0,0,0,NaN,0,C,216,False,0,0.920560,0.015541,False
10927,1,9989,398038886,R,MLO,60.0,0,0,0,0.0,0,C,216,True,0,0.604191,0.018051,False


In [59]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Extract the targets and predictions
targets = df_clean['cancer'].values
predictions_proba = df_clean['cancer_pred_proba'].values

pF1, thres = optimal_f1(targets,predictions_proba)

predictions = (predictions_proba > thres).astype(int)

recall = recall_score(targets, predictions)

auc = roc_auc_score(targets, predictions_proba)

print("pF1:", pF1)
print("Threshold:", thres)
print("Recall:", recall)
print("AUC:", auc)


pF1: 0.3661327231121282
Threshold: 0.45
Recall: 0.3463203463203463
AUC: 0.8556606041182597


## mean

In [60]:
def ensemble_predictions(models, df_train, folds):
    df_train_predictions = []
    for model in models:
        df_train_predictions.append(gen_predictions([model], df_train, folds))
    df_ensemble = pd.concat(df_train_predictions)
    df_ensemble_mean = df_ensemble.groupby(['patient_id', 'image_id'])['cancer_pred_proba'].mean().reset_index()
    df_ensemble_vote = df_ensemble.groupby(['patient_id', 'image_id'])['cancer_pred'].mean().reset_index()
    df_ensemble = pd.concat([df_ensemble_mean, df_ensemble_vote['cancer_pred']], axis=1)
    return df_ensemble

In [61]:
ensemble_table = ensemble_predictions(models,df_train,CFG.folds)
ensemble_table

Eval: 100%|██████████| 342/342 [05:19<00:00,  1.07it/s]


,patient_id,image_id,cancer_pred_proba,cancer_pred
0,25,822390278,0.053894,0.000000
1,25,1442180348,0.064209,0.000000
2,25,1723979573,0.143311,0.000000
3,25,1789648218,0.087830,0.000000
4,25,1962456803,0.189575,0.333333
...,...,...,...,...
10924,65482,1203545478,0.010391,0.000000
10925,65525,308255045,0.013489,0.000000
10926,65525,751035345,0.011398,0.000000
10927,65525,782942436,0.012405,0.000000


In [62]:
df_combined = pd.merge(df_train, ensemble_table, on=['patient_id', 'image_id'])

df_clean = df_combined.dropna(subset=['cancer_pred_proba'])
df_clean

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,split,sample_rand,cancer_pred_proba,cancer_pred
0,2,10048,964141995,L,MLO,62.0,0,0,0,NaN,0,NaN,29,False,0,0.892034,0.009384,0.0
1,2,10048,1234933874,L,CC,62.0,0,0,0,NaN,0,NaN,29,False,0,0.111358,0.011971,0.0
2,2,10048,1577142909,R,MLO,62.0,0,0,0,NaN,0,NaN,29,False,0,0.196719,0.008041,0.0
3,2,10048,1842203124,R,CC,62.0,0,0,0,NaN,0,NaN,29,False,0,0.387039,0.008652,0.0
4,1,10049,349510516,L,CC,52.0,0,0,0,NaN,0,C,49,False,0,0.219363,0.011238,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10924,1,997,1775983513,R,MLO,42.0,0,0,0,1.0,0,B,49,False,0,0.023726,0.028473,0.0
10925,1,9989,63473691,L,MLO,60.0,0,0,0,NaN,0,C,216,False,0,0.856387,0.007473,0.0
10926,1,9989,1078943060,L,CC,60.0,0,0,0,NaN,0,C,216,False,0,0.920560,0.009323,0.0
10927,1,9989,398038886,R,MLO,60.0,0,0,0,0.0,0,C,216,True,0,0.604191,0.009308,0.0


In [63]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Extract the targets and predictions
targets = df_clean['cancer'].values
predictions_proba = df_clean['cancer_pred_proba'].values

pF1, thres = optimal_f1(targets,predictions_proba)

predictions = (predictions_proba > thres).astype(int)

recall = recall_score(targets, predictions)

auc = roc_auc_score(targets, predictions_proba)

print("pF1:", pF1)
print("Threshold:", thres)
print("Recall:", recall)
print("AUC:", auc)


pF1: 0.38190954773869346
Threshold: 0.34
Recall: 0.329004329004329
AUC: 0.8578042665255229
